<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_7/lightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831166 sha256=99590899fbc53575825b7e697d2a75d5a5ad7e719002a7858e5e63a0eb8a5303
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install pandas-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.6/262.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 63.6 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=81e11425985aa8e4ffa2e93e9df6ef46382e96073b1522d3f183be098c603466
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from IPython.display import display_html
import warnings

from matplotlib.gridspec import GridSpec
%matplotlib inline

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset
from skopt import forest_minimize


In [ ]:
def display_side_by_side(*args):
  html_str = ''
  for df in args:
    html_str += df.to_html()
  display_html(html_str.replace(
    'table', 'table style="display:inline"'), raw=True)

# update the working directory to the root of the project
os.chdir('..')
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path ='content/drive/MyDrive/Recommender_systems'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%time
book_metadata = pd.read_json(path+'/goodreads_books_poetry.json', lines=True)
interaction = pd.read_json(path+'/goodreads_interaction_poetry.json', lines=True)


In [ ]:
book_metadata.columns.values

array(['isbn', 'text_reviews_count', 'series', 'countru_code', 'language_code', ' popular_shelves', 'asin',  'is_ebook', 'average_rating', ' kindle_asin', 'similar_books', ' description', 'format', 'link',  'authors', 'publisher', 'num_pages', 'publication_day', 'isbn13', 'publication_month', 'edition_information', 'publication_year', 'url', 'image_url', 'book_id', 'ratings_count', 'work_id', title', title_without_series'], dtype=object)

In [ ]:
book_metadata.sample(2)

In [ ]:
books_metadata_selected = book_metadata[['book_id', 'average_rating', 'is_ebook', 'num_pages', 'publication_year', 'ratings_count', 'language_code']]
books_metadata_selected.sample(5)

In [ ]:
import pandas_profiling as pp

# replace the blank cell with Nan
book_metadata_selected.replace('', np.nan, inplace=True)

# not taking book_id into profiler report
profile = pp.ProfileReport(book_metadata_selected[['average_rating', 'is_ebook', 'num_pages', 'publication_year', 'ratings_count']])
profile.to_file(output_file=path +'/result/profiler_books_metadata_1.html')

In [ ]:
profile

In [ ]:
# using pandas cut method to c onvert fields into discrete intervals
books_metadata_selected['num_pages'].replace(np.nan, -1, inplace=True)
books_metadata_selected['num_pages'] = pd.to_numeric(books_metadata_selected['num_pages'])
books_metadata_selected['num_pages'] = pd.cut(books_metadata_selected['num_pages'], bins=25)

# rounding ratings to neares .5 score
books_metadata_selected['average_rating'] = books_metadata_selected['average_rating'].apply(lambda x: round(x * 2) / 2)

# using pandas qcut method to convert  fields into quantile-based discrete intervals
books_metadata_selected['ratings_count'] = pd.to_numeric(books_metadata_selected['ratings_count'])
books_metadata_selected['ratings_count'] = pd.qcut(books_metadata_selected['ratings_count'], 25)

# replacing missing values to year 2100
books_metadata_selected['publication_year'].replace(np.nan, 2100, inplace=True)

# replacing missing values to'unknoun'
books_metadata_selected['language_code'].replace(np.nan, 'unknown', inplace=True)

# convert is_ebook into 1/0 where true=1 and false=0
books_metadata_selected['is_ebook'] = books_metadata_selected['is_ebook'].apply(lambda x: 1.0*(x == 'true')) # lambda x: 1 if x else 0

In [ ]:
porofile = pandas_profiling.ProfileReport(books_metadata_selected[['average_rating', 'is_ebook', 'num_pages', 'publication_year', 'ratings_count']])
profile.to_file(output_file=path +'/result/profiler_books_metadata_2.html')

In [ ]:
profile

In [ ]:
books_metadata_selected.sample(5)

In [ ]:
interaction.column.values

array(['user_id', 'book_id', 'review_id', 'is_read', 'rating', 'review_text_incomplite', 'data_added', 'data_updated', 'read_at', 'started_at'], dtype=object)

In [ ]:
interaction.sample(5)

In [ ]:
interaction.shape

In [ ]:
# limit the books metadata to selected fields
interactions_selected = interactions[['user_id', 'book_id', 'review_id', 'is_read', 'rating']]

# mapping boolean to string
booleanDictionary = {True: 'true', False: 'false'}
interactions_selected['is_read'] = interaction_selected['is_read'].replace(booleanDictionary)

interactions_selected.sample(5)

In [ ]:
profile = pandas_profiling.ProfileReport(interactions_selected[['is_read', 'rating']])
profile.to_file(output_file=path +'/result/profiler_interactions.html')

In [ ]:
# convert is_read column into 1/0 where true=1 and false=0
interactions_selected['is_read'] = interactions_selected['is_read'].apply(lambda x: 1.0*(x == 'true')) # lambda x: 1 if x else 0

In [ ]:
interactions_selected.sample(10)

In [ ]:
interactions_selected.groupby(['rating', 'is_read']).size().reset_index().pivot(columns='rating', index='is_read', values=0)

In [ ]:
import random

interactions_selected = interactions_selected.loc[interactions_selected['is_read']==1, ['user_id', 'book_id', 'rating']]
interactions_selected = interactions_selected[interactions_selected['user_id'].isin(random.sample(list(interactions_selected['user_id'].unique()), k=5000))]

interactions_selected.sample(10)

In [ ]:
interactions_selected.shape

In [ ]:
item_dict = {}
df = books_metadata[['books_id', 'title']].sort_values('book_id').reset_index()
for i in range(df.shape[0]):
  item_dict[(df.loc[i, 'book_id'])] = df.loc[i, 'title']

In [ ]:
# dummify categorical features
books_metadata_selected_transformed = pd.get_dummies(books_metadata_selected, columns=['average_rating', 'is_ebook', 'num_pages', 'publication_year', 'ratings_count', 'language_code'])
books_metadata_selected_transformed = books_metadata_selected_transformed.sort_values('book_id').reset_index().drop('index', axis=1)
books_metadata_selected_transformed.head(5)

In [ ]:
# convert to csr matrix
books_metadata_csr = csr_matrix(books_metadata_selected_transformed.drop('book_id', axis=1).values)
books_metadata_csr

In [ ]:
user_book_interaction = pd.pivot_table(interactions_selected, index='user_id', columns='book_id', values='rating')

#fill missing values with 0
user_book_interaction = user_book_interaction.fillna(0)
user_book_interaction.head(10)

In [ ]:
user_id = list(user_book_interaction.index)
user_dict = {}
counter = 0
for i in user_id:
  user_dict[i] = counter
  counter += 1

In [ ]:
# convert to csr matrix
user_book_interaction_csr = csr_matrix(user_book_interaction.values)
user_book_interaction_csr

In [ ]:
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(user_book_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)


In [ ]:
def sample_recomendation_user(model, interactions, user_id, user_dict, item_dict, threshold=0, nrec_items=5, show=True):

  n_users, n_irtems = interactions.shape
  user_x = user_dict[user_id]
  scores = pd.Series(model.predict(user_x, np.arange(n_items), item_features=books_metadata_csr))
  scores.index = interactions.columns
  scores = list(pd.Series(scores.sort_values(ascending=False).index))

  known_items = list(pd.Series(interactions.loc[user_id, :]
                               [interactions.loc[user_id, :] > threshold].index)
                               .sort_values(ascending=False))

  scores = [x for x in scores if x not in known_items]
  return_score_list = scores[0:nrec_items]
  known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
  scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))

  if show == True:
    print("User: " + str(user_id))
    print("Known Likes:")
    counter = 1
    for i in known_items:
      print(str(counter) + ': ' + i)
      counter += 1

    print("\n Recommended Items:")
    counter = 1
    for i in scores:
      print(str(counter) + ': ' + i)
      counter += 1


In [ ]:
sample_recomendation_user(model, user_book_interaction, 'ff52b7331f2ccab0582678644fed9d85', user_dict, item_dict)

In [ ]:
sample_recomendation_user(model, user_book_interaction, 'ff52g7331f2ccab0682678644fed9f85', user_dict, item_dict)